# Neutralizing bad datapoints

Once you have found dummy values (such as 9999), bad datapoints or outliers, you will want to neutralize them in some way.

You have about three options. Most often, the first option is best.

* Replace the bad values with a dummy value (such as `NaN`)
    - This retains as much information as possible
*  Replace the whole record (row of the dataframe) with a dummy value (such as `NaN`)
    - Useful if the bad value for one variable casts doubt on the data quality for the whole record, or for some reason you need to retain only complete records
   
* Delete the whole record (row of the dataframe)
    - Generally not recommended as could be seen as dishonest (see below)
    - Exception would be obvious duplicate records or completely blank records

### Set up Python Libraries

As usual you will need to run this code block to import the relevant Python libraries

In [1]:
# Set-up Python libraries - you need to run this but you don't need to change it
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import seaborn as sns
sns.set_theme(style='white')
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Import a dataset to work with

You will need to download the file heartAttack.csv from Canvas to your computer, then import it

In [2]:
hospital=pd.read_csv('https://raw.githubusercontent.com/jillxoreilly/StatsCourseBook_2024/main/data/heartAttack.csv')
display(hospital)

HTTPError: HTTP Error 404: Not Found

## Replace only the bad values

- `df.replace(old_value, new_value, inplace=True)`
- `df.loc[row_index, column_index] = new_value`

In most cases the best option is to replace only the bad values, retaining the rest of the record.

We will replace the values with `NaN` - 'not a number' which is a dummy value that will be ignored by most Python functions (for example, if we calculate the mean of a column containing NaNs, Pandas just calculates the mean of all the non-NaN values)

### Replace a dummy value with `df.replace()`

If all the bad datapoints have the same value, for example <tt>9999</tt>, we can easily replace them as follows: 

In [3]:
# note use argument inplace=True to edit the original dataframe
hospital.LOS.replace(9999, np.NaN, inplace=True) 
hospital.AGE.replace(9999, np.NaN, inplace=True) 

# check they have gone - max values for LOS and AGE should no longer be 9999
hospital.describe()

,CHARGES,LOS,AGE,DRG,DIED
count,12145.000000,12843.000000,12841.000000,12841.000000,12841.000000
mean,9879.087615,7.567858,66.343275,121.690523,0.109805
std,6558.399650,5.114357,15.014263,0.658289,0.312658
min,3.000000,0.000000,20.000000,121.000000,0.000000
25%,5422.200000,4.000000,57.000000,121.000000,0.000000
50%,8445.000000,7.000000,67.000000,122.000000,0.000000
75%,12569.040000,10.000000,77.000000,122.000000,0.000000
max,47910.120000,38.000000,774.000000,123.000000,1.000000


### Replace values over a cutoff with df.loc[]

In some cases we want to replace a range of values; say for example we decided to remove charges over 30000 dollars.

To do this we unfortunately have to use different syntax from our regular indexing with `df.query()`, as we are *setting* values in the dataframe. We use the function `df.loc[]` which accesses specific *locations* in the dataframe defined by row and column numbers.

Say if I wanted to replace the value of <tt>SEX</tt> in the first row to <tt>'bananas'</tt> (!):

In [4]:
hospital.loc[0,'SEX']='bananas' # remember row zero is the first row in Python!
hospital.head() # check it worked

,CHARGES,LOS,AGE,SEX,DRG,DIED
0,4752.0,10.0,79.0,bananas,122.0,0.0
1,3941.0,6.0,34.0,F,122.0,0.0
2,3657.0,5.0,76.0,F,122.0,0.0
3,1481.0,2.0,80.0,F,122.0,0.0
4,1681.0,1.0,55.0,M,122.0,0.0


Now, instead of giving a row number, I can use a criterion (<tt>hospital.CHARGES > 30000</tt>) to find the rows I want to edit:

In [5]:
hospital.loc[(hospital.CHARGES > 30000),'CHARGES']=np.nan # remember row zero is the first row in Python!
hospital.describe() # check it worked

,CHARGES,LOS,AGE,DRG,DIED
count,11928.000000,12843.000000,12841.000000,12841.000000,12841.000000
mean,9402.119925,7.567858,66.343275,121.690523,0.109805
std,5537.499427,5.114357,15.014263,0.658289,0.312658
min,3.000000,0.000000,20.000000,121.000000,0.000000
25%,5369.150000,4.000000,57.000000,121.000000,0.000000
50%,8329.775000,7.000000,67.000000,122.000000,0.000000
75%,12278.375000,10.000000,77.000000,122.000000,0.000000
max,30000.000000,38.000000,774.000000,123.000000,1.000000


* **Note** the maximum value of CHARGES should now be 30000 dollars

### Replace the whole record

Occasionally we decide a whole record or row of the data table (often corresponding to an individual) should be replaced with `NaN`.

Some situations in which we would replace the whole record with `NaN` would be:

* We realise after the fact that a participant didn't meet includion criteria for our study - for example, we are studying unmedicated patients witha  certain condition, and they disclosed after data collection that they are already on medication
* They have bad values for several variables
* we wish to only retain records that are complete

We can replace the whole record with `NaN` using `df.loc[]` and simply not specifying a column. For example, let's make the whole second row of the dataframe become `NaN`

In [6]:
hospital.loc[1] = np.nan # remember we count from zero in Python so second row = row 1 !
hospital.head() # check it worked

,CHARGES,LOS,AGE,SEX,DRG,DIED
0,4752.0,10.0,79.0,bananas,122.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN
2,3657.0,5.0,76.0,F,122.0,0.0
3,1481.0,2.0,80.0,F,122.0,0.0
4,1681.0,1.0,55.0,M,122.0,0.0


Or say we want to replace the whole record for anyone with <tt>CHARGES</tt> under 100 dollars with `NaN`:

In [7]:
hospital.loc[(hospital.CHARGES<100)] = np.nan # remember we count from zero in Python so second row = row 1 !
hospital.describe() # check it worked - mmin charge should now be >= $100

,CHARGES,LOS,AGE,DRG,DIED
count,11919.000000,12834.000000,12832.000000,12832.000000,12832.000000
mean,9408.858857,7.568568,66.349205,121.689994,0.109570
std,5534.051880,5.113899,15.012255,0.658085,0.312365
min,101.000000,0.000000,20.000000,121.000000,0.000000
25%,5374.960000,4.000000,57.000000,121.000000,0.000000
50%,8334.000000,7.000000,67.000000,122.000000,0.000000
75%,12285.550000,10.000000,77.000000,122.000000,0.000000
max,30000.000000,38.000000,774.000000,123.000000,1.000000


### Drop the record from the dataframe

* `df.drop(row_index)`

**Generally not recommended** - in general we replace missing data with `NaN` so that it is transparent how many bad datapoints were collected. 

Imagine if we conducted a survey including a question "have you ever committed a crime". Perhaps some people would not answer this question (mmaybe momre likely if the answer is 'yes'). 

If we deleted all the records for people who skipped the question, this could be misleading.

**However** sometimes we need to just remove rows. An example would be if a pile of forms were automatically scanned, and it turned out a lot of the were completely blank (spare forms). We would want to delete the records corresponding to those spare forms.

Another example would be if a record was an exact duplicate of another.

To remove records (rows) we use `df.drop()`. 

For example, perhaps we want to remove row 0 (the first row), as we this it is a prank entry (<tt>SEX == 'bananas'</tt>):

In [8]:
hospital.drop(0, inplace=True)
hospital # check it's gone

,CHARGES,LOS,AGE,SEX,DRG,DIED
1,NaN,NaN,NaN,NaN,NaN,NaN
2,3657.00,5.0,76.0,F,122.0,0.0
3,1481.00,2.0,80.0,F,122.0,0.0
4,1681.00,1.0,55.0,M,122.0,0.0
5,6378.64,9.0,84.0,M,121.0,0.0
...,...,...,...,...,...,...
12839,22603.57,14.0,79.0,F,121.0,0.0
12840,NaN,7.0,91.0,F,121.0,0.0
12841,14359.14,9.0,79.0,F,121.0,0.0
12842,12986.00,5.0,70.0,M,121.0,0.0


### Exercises

Complete the following exercises to get used to the syntax for replacing values.
It's boring but if you master it now it will save some stress later on.

In [17]:
# 1. For all cases where AGE is greater than 120, replace the value of AGE with `NaN`
# Reload the data:
hospital=pd.read_csv('https://raw.githubusercontent.com/jillxoreilly/StatsCourseBook_2024/main/data/heartAttack.csv')

# YOUR CODE HERE

# uncomment the next line to check it worked
hospital.sort_values(by='AGE')

,CHARGES,LOS,AGE,SEX,DRG,DIED
5410,6214.00,4,20.0,M,122.0,0.0
10852,6726.27,4,21.0,F,122.0,0.0
10737,NaN,8,23.0,M,121.0,0.0
4125,10781.00,8,23.0,M,122.0,0.0
5198,7596.00,8,24.0,M,121.0,0.0
...,...,...,...,...,...,...
5066,5486.00,7,103.0,M,123.0,1.0
639,5819.72,8,NaN,F,121.0,0.0
708,13405.00,13,NaN,M,122.0,0.0
909,11648.00,9999,NaN,F,121.0,0.0


In [18]:
# 2. For all cases where AGE is greater than 120, replace the entire record with `NaN`
# Reload the data:
hospital=pd.read_csv('https://raw.githubusercontent.com/jillxoreilly/StatsCourseBook_2024/main/data/heartAttack.csv')

# YOUR CODE HERE

# uncomment the next line to check it worked
hospital.sort_values(by='AGE')

,CHARGES,LOS,AGE,SEX,DRG,DIED
5410,6214.00,4.0,20.0,M,122.0,0.0
10852,6726.27,4.0,21.0,F,122.0,0.0
10737,NaN,8.0,23.0,M,121.0,0.0
4125,10781.00,8.0,23.0,M,122.0,0.0
5198,7596.00,8.0,24.0,M,121.0,0.0
...,...,...,...,...,...,...
5066,5486.00,7.0,103.0,M,123.0,1.0
639,5819.72,8.0,NaN,F,121.0,0.0
708,13405.00,13.0,NaN,M,122.0,0.0
909,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# 3. For all values 9999 in all columns, replace the 9999 with NaN
# Reload the data:
hospital=pd.read_csv('https://raw.githubusercontent.com/jillxoreilly/StatsCourseBook_2024/main/data/heartAttack.csv')

# YOUR CODE HERE

# uncomment the next line to check it worked
# hospital.sort_values(by='LOS')

,CHARGES,LOS,AGE,SEX,DRG,DIED
9158,10366.00,0.0,53.0,F,122.0,0.0
6422,2817.00,1.0,43.0,M,122.0,0.0
5223,2202.00,1.0,76.0,M,123.0,1.0
5206,2083.50,1.0,87.0,M,123.0,1.0
5182,2283.00,1.0,83.0,F,121.0,0.0
...,...,...,...,...,...,...
5860,33236.13,38.0,82.0,F,123.0,1.0
9803,15333.00,38.0,69.0,M,123.0,1.0
1157,13961.51,38.0,83.0,F,123.0,1.0
5196,17928.00,38.0,84.0,F,122.0,0.0


In [27]:
# 4. Replace the code DRG = 123 with the string 'DIED'
# Reload the data:
hospital=pd.read_csv('https://raw.githubusercontent.com/jillxoreilly/StatsCourseBook_2024/main/data/heartAttack.csv')

# YOUR CODE HERE

# uncomment the next line to check it worked
# hospital

,CHARGES,LOS,AGE,SEX,DRG,DIED
0,4752.00,10,79.0,F,122.0,0.0
1,3941.00,6,34.0,F,122.0,0.0
2,3657.00,5,76.0,F,122.0,0.0
3,1481.00,2,80.0,F,122.0,0.0
4,1681.00,1,55.0,M,122.0,0.0
...,...,...,...,...,...,...
12839,22603.57,14,79.0,F,121.0,0.0
12840,NaN,7,91.0,F,121.0,0.0
12841,14359.14,9,79.0,F,121.0,0.0
12842,12986.00,5,70.0,M,121.0,0.0
